In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [9]:
pd.set_option('display.max_columns', 40)
pd.set_option('display.float_format', '{:.2f}'.format)

In [23]:
df = pd.read_csv("../data_drugs1.csv", encoding='1251')
df.head()

,DR_Dat,DR_Tim,DR_NChk,DR_NDoc,DR_Apt,DR_Kkm,DR_TDoc,DR_TPay,DR_CDrugs,DR_NDrugs,DR_Suppl,DR_Prod,DR_Kol,DR_CZak,DR_CRoz,DR_SDisc,DR_CDisc,DR_BCDisc,DR_TabEmpl,DR_VZak,DR_Pos
0,2022-08-01,08:06:18,1272,13002561,13,22589,Розничная реализация,18,144734,ГАСТАЛ №12 ТАБ. Д/РАСС.,Пульс,TEVA Pharvaceutical Industries Ltd,1.00,196.71,270.00,0.00,NaN,NaN,29,1,1.00
1,2022-08-01,08:38:53,1273,13002561,13,22589,Розничная реализация,15,69661,"ТОБРОПТ 0,3% 5МЛ. №1 ГЛ.КАПЛИ ФЛ./КАП. /РОМФАРМ/",Пульс,РОМФАРМ КОМПАНИ ( ROMPHARM ),1.00,106.21,127.00,6.00,9.00,200010004357.00,29,1,1.00
2,2022-08-01,08:55:38,1274,13002561,13,22589,Розничная реализация,18,190635,ЭЛИКВИС 5МГ. №60 ТАБ. П/П/О /ПФАЙЗЕР/БРИСТОЛ-М...,ГРАНД КАПИТАЛ СМОЛЕНСК ООО ФК,Пфайзер,1.00,2320.99,2563.00,76.00,9.00,200010018491.00,29,1,1.00
3,2022-08-01,09:00:40,1275,13002561,13,22589,Розничная реализация,18,276370,АРБИДОЛ МАКСИМУМ 200МГ. №10 КАПС. /ОТИСИФАРМ/Ф...,Пульс,ОТИСИФАРМ ПАО,1.00,445.39,541.00,0.00,NaN,NaN,29,1,1.00
4,2022-08-01,09:04:05,1276,13002561,13,22589,Розничная реализация,15,2303,"ЭНАМ 2,5МГ. №20 ТАБ. /Д-Р РЕДДИ/",Протек,Д-р Редди с Лабораторис Лтд / Dr.REDDY's,1.00,18.04,22.00,1.00,9.00,200010000734.00,29,1,5.00


# Динамика продаж

## Rolling

In [24]:
df.DR_Dat.unique()

array(['2022-08-01', '2022-08-02', '2022-08-03', '2022-08-04',
       '2022-08-05'], dtype=object)

In [47]:
df_revenue = df.groupby('DR_Dat').apply(lambda row: pd.Series({'revenue':sum(row['DR_Kol'] * row['DR_CRoz'] - row['DR_SDisc'])}))
df_revenue

,revenue
DR_Dat,
2022-08-01,84681.52
2022-08-02,71389.37
2022-08-03,78050.82
2022-08-04,59187.36
2022-08-05,56458.81


In [48]:
df_revenue['d1'] = 1
df_revenue['d1'] = df_revenue['revenue'].rolling(2, min_periods=1).apply(lambda x: ((x.iloc[1] - x.iloc[0]) / x.iloc[0]) * 100 if len(x) == 2 else 0)
# df_revenue
# df_revenue[].rolling(2, min_periods=1).apply(lambda x: ((x.iloc[1] - x.iloc[0]) / x.iloc[0]) * 100 if len(x) == 2 else 0)
df_revenue = df_revenue.reset_index()
df_revenue

,DR_Dat,revenue,d1
0,2022-08-01,84681.52,0.00
1,2022-08-02,71389.37,-15.70
2,2022-08-03,78050.82,9.33
3,2022-08-04,59187.36,-24.17
4,2022-08-05,56458.81,-4.61


## Shift

In [58]:
df_revenue['revenue_shifted'] = df_revenue["revenue"].shift(1)
df_revenue['d2'] = ((df_revenue['revenue'] - df_revenue['revenue_shifted']) / df_revenue['revenue_shifted']) * 100
df_revenue

,DR_Dat,revenue,d1,d2,d3,revenue_shifted
0,2022-08-01,84681.52,0.00,NaN,NaN,NaN
1,2022-08-02,71389.37,-15.70,-15.70,-15.70,84681.52
2,2022-08-03,78050.82,9.33,9.33,9.33,71389.37
3,2022-08-04,59187.36,-24.17,-24.17,-24.17,78050.82
4,2022-08-05,56458.81,-4.61,-4.61,-4.61,59187.36


### pct_change

In [67]:
df_revenue['d3'] = df_revenue['revenue'].pct_change(1)*100
df_revenue = df_revenue.drop(columns=['revenue_shifted']) if 'revenue_shifted' in df_revenue.columns else df_revenue
df_revenue.columns = ['DR_Dat', 'revenue', 'd1', 'd2', 'd3']
df_revenue.fillna(0)
df_revenue

,DR_Dat,revenue,d1,d2,d3
0,2022-08-01,84681.52,0.00,NaN,NaN
1,2022-08-02,71389.37,-15.70,-15.70,-15.70
2,2022-08-03,78050.82,9.33,9.33,9.33
3,2022-08-04,59187.36,-24.17,-24.17,-24.17
4,2022-08-05,56458.81,-4.61,-4.61,-4.61
